In [3]:
import pandas as pd
import numpy as np
import urllib.request, json, csv 
from bs4 import BeautifulSoup
import requests
import boto3
from urllib.request import urlopen
from zipfile import ZipFile
import shapefile

In [4]:
url = 'https://www2.census.gov/geo/pvs/tiger2010st/'
response = requests.get(url)

In [5]:
soup = BeautifulSoup(response.text)

In [6]:
states = [state.text[:2] for state in soup.find('table').find_all('a')[5:56]]

In [9]:
states

['01',
 '02',
 '04',
 '05',
 '06',
 '08',
 '09',
 '10',
 '11',
 '12',
 '13',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '53',
 '54',
 '55',
 '56']

In [17]:
len(states)

51

In [10]:
! rm data/*csv

In [11]:
! ls data

tl_2010_06_tract10.dbf	tl_2010_06_tract10.shp	    tl_2010_06_tract10.shx
tl_2010_06_tract10.prj	tl_2010_06_tract10.shp.xml  tl_2010_06_tract10.zip


In [12]:
base_url = 'https://api.census.gov/data/'
api_request = '/acs/acs5?get=B19013_001E,B19301_001E,B15002_019E,B15002_032E,B15002_033E,B15002_034E,B15002_035E,B15002_002E,B15002_015E,B15002_016E,B15002_017E,B15002_018E,B08301_001E,B08301_003E,B08301_004E,B08301_010E,B08301_019E,B08301_021E,B02001_001E,B02001_002E,B02001_003E,B01002_001E,B01003_001E,GEO_ID,NAME&for=tract:*&in=county:*&in=state:'
for year in range(2010, 2019):
    for state in states:
        census_url = base_url+str(year)+api_request+state
        with urllib.request.urlopen(census_url) as url:
            data = json.loads(url.read().decode())
        data[0].append('year')
        for row in data[1:]:
            row.append(int(year))
        with open('data/census_'+state+'_'+str(year)+'.csv', 'w') as outfile:
            writer = csv.writer(outfile)
            writer.writerows(data)

In [16]:
! ls data/*csv

data/census_01_2010.csv  data/census_21_2010.csv  data/census_38_2010.csv
data/census_01_2011.csv  data/census_21_2011.csv  data/census_38_2011.csv
data/census_01_2012.csv  data/census_21_2012.csv  data/census_38_2012.csv
data/census_01_2013.csv  data/census_21_2013.csv  data/census_38_2013.csv
data/census_01_2014.csv  data/census_21_2014.csv  data/census_38_2014.csv
data/census_01_2015.csv  data/census_21_2015.csv  data/census_38_2015.csv
data/census_01_2016.csv  data/census_21_2016.csv  data/census_38_2016.csv
data/census_01_2017.csv  data/census_21_2017.csv  data/census_38_2017.csv
data/census_01_2018.csv  data/census_21_2018.csv  data/census_38_2018.csv
data/census_02_2010.csv  data/census_22_2010.csv  data/census_39_2010.csv
data/census_02_2011.csv  data/census_22_2011.csv  data/census_39_2011.csv
data/census_02_2012.csv  data/census_22_2012.csv  data/census_39_2012.csv
data/census_02_2013.csv  data/census_22_2013.csv  data/census_39_2013.csv
data/census_02_2014.csv  data/census_2

In [19]:
s3_resource = boto3.resource('s3')

In [20]:
bucket = s3_resource.Bucket(name='masonedefault')

In [23]:
file_name = 'data/census_56_2018.csv'

In [27]:
for year in range(2010, 2019):
    for state in states:
        file_name = 'data/census_'+state+'_'+str(year)+'.csv'
        bucket.upload_file(file_name, file_name[5:])
        

In [29]:
state_urls = [state.text for state in soup.find('table').find_all('a')[5:56]]

In [35]:
state_urls

['01_Alabama/',
 '02_Alaska/',
 '04_Arizona/',
 '05_Arkansas/',
 '06_California/',
 '08_Colorado/',
 '09_Connecticut/',
 '10_Delaware/',
 '11_District_of_Columbia/',
 '12_Florida/',
 '13_Georgia/',
 '15_Hawaii/',
 '16_Idaho/',
 '17_Illinois/',
 '18_Indiana/',
 '19_Iowa/',
 '20_Kansas/',
 '21_Kentucky/',
 '22_Louisiana/',
 '23_Maine/',
 '24_Maryland/',
 '25_Massachusetts/',
 '26_Michigan/',
 '27_Minnesota/',
 '28_Mississippi/',
 '29_Missouri/',
 '30_Montana/',
 '31_Nebraska/',
 '32_Nevada/',
 '33_New_Hampshire/',
 '34_New_Jersey/',
 '35_New_Mexico/',
 '36_New_York/',
 '37_North_Carolina/',
 '38_North_Dakota/',
 '39_Ohio/',
 '40_Oklahoma/',
 '41_Oregon/',
 '42_Pennsylvania/',
 '44_Rhode_Island/',
 '45_South_Carolina/',
 '46_South_Dakota/',
 '47_Tennessee/',
 '48_Texas/',
 '49_Utah/',
 '50_Vermont/',
 '51_Virginia/',
 '53_Washington/',
 '54_West_Virginia/',
 '55_Wisconsin/',
 '56_Wyoming/']

In [33]:
url = 'https://www2.census.gov/geo/pvs/tiger2010st/'
response = requests.get(url)

soup = BeautifulSoup(response.text)

In [38]:
url+state_urls[0]+state_urls[0][:2]+'/tl_2010_'+state_urls[0][:2]+'_tract10.zip'

'https://www2.census.gov/geo/pvs/tiger2010st/01_Alabama/01/tl_2010_01_tract10.zip'

In [39]:
! rm data/tl*

In [40]:
for state in state_urls:
    zip_url = url+state+state[:2]+'/tl_2010_'+state[:2]+'_tract10.zip'
    zip_file = urlopen(zip_url)

    with open('/tmp/tempfile.zip', 'wb') as tmp:
        tmp.write(zip_file.read())

    with ZipFile('/tmp/tempfile.zip', 'r') as zip_ref:
        zip_ref.extractall('data/')

    sf = shapefile.Reader('data/tl_2010_'+state[:2]+'_tract10.shp')
    geodata = [list(record) for record in sf.records()]
    
    
    with open('data/geo_'+state[:-1]+'.csv', 'w') as outfile:
            writer = csv.writer(outfile)
            writer.writerow([x[0] for x in sf.fields[1:]])
            writer.writerows(geodata)
    
    ! rm data/tl*

In [41]:
! ls data/geo*

data/geo_01_Alabama.csv		      data/geo_30_Montana.csv
data/geo_02_Alaska.csv		      data/geo_31_Nebraska.csv
data/geo_04_Arizona.csv		      data/geo_32_Nevada.csv
data/geo_05_Arkansas.csv	      data/geo_33_New_Hampshire.csv
data/geo_06_California.csv	      data/geo_34_New_Jersey.csv
data/geo_08_Colorado.csv	      data/geo_35_New_Mexico.csv
data/geo_09_Connecticut.csv	      data/geo_36_New_York.csv
data/geo_10_Delaware.csv	      data/geo_37_North_Carolina.csv
data/geo_11_District_of_Columbia.csv  data/geo_38_North_Dakota.csv
data/geo_12_Florida.csv		      data/geo_39_Ohio.csv
data/geo_13_Georgia.csv		      data/geo_40_Oklahoma.csv
data/geo_15_Hawaii.csv		      data/geo_41_Oregon.csv
data/geo_16_Idaho.csv		      data/geo_42_Pennsylvania.csv
data/geo_17_Illinois.csv	      data/geo_44_Rhode_Island.csv
data/geo_18_Indiana.csv		      data/geo_45_South_Carolina.csv
data/geo_19_Iowa.csv		      data/geo_46_South_Dakota.csv
data/geo_20_Kansas.csv		      data/geo_47_Tennessee.csv
data/geo_21_Ken

In [43]:
'data/geo_'+state_urls[0][:-1]+'.csv'

'data/geo_01_Alabama.csv'

In [44]:
for state in state_urls:
    file_name = 'data/geo_'+state[:-1]+'.csv'
    bucket.upload_file(file_name, file_name[5:])
        

In [2]:
import pyspark

spark = pyspark.sql.SparkSession.builder.config('spark.driver.memory', '3g').getOrCreate()

In [45]:
! ls data/census*.csv

data/census_01_2010.csv  data/census_21_2010.csv  data/census_38_2010.csv
data/census_01_2011.csv  data/census_21_2011.csv  data/census_38_2011.csv
data/census_01_2012.csv  data/census_21_2012.csv  data/census_38_2012.csv
data/census_01_2013.csv  data/census_21_2013.csv  data/census_38_2013.csv
data/census_01_2014.csv  data/census_21_2014.csv  data/census_38_2014.csv
data/census_01_2015.csv  data/census_21_2015.csv  data/census_38_2015.csv
data/census_01_2016.csv  data/census_21_2016.csv  data/census_38_2016.csv
data/census_01_2017.csv  data/census_21_2017.csv  data/census_38_2017.csv
data/census_01_2018.csv  data/census_21_2018.csv  data/census_38_2018.csv
data/census_02_2010.csv  data/census_22_2010.csv  data/census_39_2010.csv
data/census_02_2011.csv  data/census_22_2011.csv  data/census_39_2011.csv
data/census_02_2012.csv  data/census_22_2012.csv  data/census_39_2012.csv
data/census_02_2013.csv  data/census_22_2013.csv  data/census_39_2013.csv
data/census_02_2014.csv  data/census_2

In [4]:
census = spark.read.csv('data/census*.csv', header='true', inferSchema='true')

In [5]:
census.count()

+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+--------------------+--------------------+-----+------+-----+----+
|B19013_001E|B19301_001E|B15002_019E|B15002_032E|B15002_033E|B15002_034E|B15002_035E|B15002_002E|B15002_015E|B15002_016E|B15002_017E|B15002_018E|B08301_001E|B08301_003E|B08301_004E|B08301_010E|B08301_019E|B08301_021E|B02001_001E|B02001_002E|B02001_003E|B01002_001E|B01003_001E|              GEO_ID|                NAME|state|county|tract|year|
+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+--------------------+--------------

In [6]:
from pyspark.sql.functions import split, explode, col, lower, sort_array, expr

In [7]:
census.printSchema()

root
 |-- B19013_001E: integer (nullable = true)
 |-- B19301_001E: integer (nullable = true)
 |-- B15002_019E: integer (nullable = true)
 |-- B15002_032E: integer (nullable = true)
 |-- B15002_033E: integer (nullable = true)
 |-- B15002_034E: integer (nullable = true)
 |-- B15002_035E: integer (nullable = true)
 |-- B15002_002E: integer (nullable = true)
 |-- B15002_015E: integer (nullable = true)
 |-- B15002_016E: integer (nullable = true)
 |-- B15002_017E: integer (nullable = true)
 |-- B15002_018E: integer (nullable = true)
 |-- B08301_001E: integer (nullable = true)
 |-- B08301_003E: integer (nullable = true)
 |-- B08301_004E: integer (nullable = true)
 |-- B08301_010E: integer (nullable = true)
 |-- B08301_019E: integer (nullable = true)
 |-- B08301_021E: integer (nullable = true)
 |-- B02001_001E: integer (nullable = true)
 |-- B02001_002E: integer (nullable = true)
 |-- B02001_003E: integer (nullable = true)
 |-- B01002_001E: double (nullable = true)
 |-- B01003_001E: integer (n

In [8]:
census = census.withColumn('GEO_ID', split('GEO_ID', 'US')[1])

In [9]:
census.show(5)

+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+--------------------+-----+------+-----+----+
|B19013_001E|B19301_001E|B15002_019E|B15002_032E|B15002_033E|B15002_034E|B15002_035E|B15002_002E|B15002_015E|B15002_016E|B15002_017E|B15002_018E|B08301_001E|B08301_003E|B08301_004E|B08301_010E|B08301_019E|B08301_021E|B02001_001E|B02001_002E|B02001_003E|B01002_001E|B01003_001E|     GEO_ID|                NAME|state|county|tract|year|
+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+--------------------+-----+------+-----+-

In [10]:
census.printSchema()

root
 |-- B19013_001E: integer (nullable = true)
 |-- B19301_001E: integer (nullable = true)
 |-- B15002_019E: integer (nullable = true)
 |-- B15002_032E: integer (nullable = true)
 |-- B15002_033E: integer (nullable = true)
 |-- B15002_034E: integer (nullable = true)
 |-- B15002_035E: integer (nullable = true)
 |-- B15002_002E: integer (nullable = true)
 |-- B15002_015E: integer (nullable = true)
 |-- B15002_016E: integer (nullable = true)
 |-- B15002_017E: integer (nullable = true)
 |-- B15002_018E: integer (nullable = true)
 |-- B08301_001E: integer (nullable = true)
 |-- B08301_003E: integer (nullable = true)
 |-- B08301_004E: integer (nullable = true)
 |-- B08301_010E: integer (nullable = true)
 |-- B08301_019E: integer (nullable = true)
 |-- B08301_021E: integer (nullable = true)
 |-- B02001_001E: integer (nullable = true)
 |-- B02001_002E: integer (nullable = true)
 |-- B02001_003E: integer (nullable = true)
 |-- B01002_001E: double (nullable = true)
 |-- B01003_001E: integer (n

In [11]:
census.count()

116776

In [12]:
census = census.na.drop()

In [13]:
census = census.filter(census.B19013_001E > 0)

In [14]:
census = census.withColumn('ed_total', sum([census.B15002_032E,
                                   census.B15002_033E,
                                   census.B15002_034E,
                                   census.B15002_035E,
                                   census.B15002_015E,
                                   census.B15002_016E,
                                   census.B15002_017E,
                                   census.B15002_018E])/sum([census.B15002_019E,census.B15002_002E]))

census = census.withColumn('ed_fem', sum([census.B15002_032E,
                                 census.B15002_033E,
                                 census.B15002_034E,
                                 census.B15002_035E]) / census.B15002_019E)

census = census.withColumn('ed_male', sum([census.B15002_015E,
                                  census.B15002_016E,
                                  census.B15002_017E,
                                  census.B15002_018E]) / census.B15002_002E)

census = census.withColumn('trans_solo', census.B08301_003E/census.B08301_001E)

census = census.withColumn('trans_pool', census.B08301_004E/census.B08301_001E)

census = census.withColumn('trans_public', census.B08301_010E/census.B08301_001E)

census = census.withColumn('trans_walk', census.B08301_019E/census.B08301_001E)

census = census.withColumn('trans_remote', census.B08301_021E/census.B08301_001E)

census = census.withColumn('perc_white', census.B02001_002E/census.B02001_001E)

census = census.withColumn('perc_black', census.B02001_003E/census.B02001_001E)

census = census.withColumn('med_hous_inc', census.B19013_001E)

census = census.withColumn('per_cap_inc', census.B19301_001E)

census = census.withColumn('med_age', census.B01002_001E)

census = census.withColumn('population', census.B01003_001E)

In [15]:
census.printSchema()

root
 |-- B19013_001E: integer (nullable = true)
 |-- B19301_001E: integer (nullable = true)
 |-- B15002_019E: integer (nullable = true)
 |-- B15002_032E: integer (nullable = true)
 |-- B15002_033E: integer (nullable = true)
 |-- B15002_034E: integer (nullable = true)
 |-- B15002_035E: integer (nullable = true)
 |-- B15002_002E: integer (nullable = true)
 |-- B15002_015E: integer (nullable = true)
 |-- B15002_016E: integer (nullable = true)
 |-- B15002_017E: integer (nullable = true)
 |-- B15002_018E: integer (nullable = true)
 |-- B08301_001E: integer (nullable = true)
 |-- B08301_003E: integer (nullable = true)
 |-- B08301_004E: integer (nullable = true)
 |-- B08301_010E: integer (nullable = true)
 |-- B08301_019E: integer (nullable = true)
 |-- B08301_021E: integer (nullable = true)
 |-- B02001_001E: integer (nullable = true)
 |-- B02001_002E: integer (nullable = true)
 |-- B02001_003E: integer (nullable = true)
 |-- B01002_001E: double (nullable = true)
 |-- B01003_001E: integer (n

In [16]:
census = census.select(['GEO_ID', 'NAME', 'state', 'county', 'tract', 'year', 'ed_total', 'ed_fem', 
              'ed_male', 'trans_solo', 'trans_pool', 'trans_public', 'trans_walk', 'trans_remote',
              'perc_white', 'perc_black', 'med_hous_inc', 'per_cap_inc', 'med_age', 'population'])

In [17]:
census.printSchema()

root
 |-- GEO_ID: string (nullable = true)
 |-- NAME: string (nullable = true)
 |-- state: integer (nullable = true)
 |-- county: integer (nullable = true)
 |-- tract: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- ed_total: double (nullable = true)
 |-- ed_fem: double (nullable = true)
 |-- ed_male: double (nullable = true)
 |-- trans_solo: double (nullable = true)
 |-- trans_pool: double (nullable = true)
 |-- trans_public: double (nullable = true)
 |-- trans_walk: double (nullable = true)
 |-- trans_remote: double (nullable = true)
 |-- perc_white: double (nullable = true)
 |-- perc_black: double (nullable = true)
 |-- med_hous_inc: integer (nullable = true)
 |-- per_cap_inc: integer (nullable = true)
 |-- med_age: double (nullable = true)
 |-- population: integer (nullable = true)



In [18]:
percentile = census.filter(census.year <= 2015).groupby(['state','year']).agg(expr('percentile(med_hous_inc, array(0.10))')[0].alias('%10'))
percentile.show()

+-----+----+------------------+
|state|year|               %10|
+-----+----+------------------+
|   36|2015|           29534.5|
|   36|2014|           29185.9|
|   36|2012|           28594.0|
|   36|2010|27895.200000000004|
|    6|2013|           32173.9|
|    6|2014|           32159.8|
|    6|2012|           32268.6|
|   36|2013|           28922.4|
|    6|2015|32148.600000000002|
|    6|2010|32831.200000000004|
|    6|2011|           32922.8|
|   36|2011|           28307.6|
+-----+----+------------------+



In [20]:
for idx, row in enumerate(percentile.collect()):

    temp =  census.filter(census.state == row['state']).filter(census.year == row['year']).filter(census.med_hous_inc <= row['%10'])
    
    if idx == 0:
        df_elig = temp
    else:
        df_elig = df_elig.union(temp)


In [21]:
census.printSchema()

root
 |-- GEO_ID: string (nullable = true)
 |-- NAME: string (nullable = true)
 |-- state: integer (nullable = true)
 |-- county: integer (nullable = true)
 |-- tract: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- ed_total: double (nullable = true)
 |-- ed_fem: double (nullable = true)
 |-- ed_male: double (nullable = true)
 |-- trans_solo: double (nullable = true)
 |-- trans_pool: double (nullable = true)
 |-- trans_public: double (nullable = true)
 |-- trans_walk: double (nullable = true)
 |-- trans_remote: double (nullable = true)
 |-- perc_white: double (nullable = true)
 |-- perc_black: double (nullable = true)
 |-- med_hous_inc: integer (nullable = true)
 |-- per_cap_inc: integer (nullable = true)
 |-- med_age: double (nullable = true)
 |-- population: integer (nullable = true)



In [22]:
df_elig.printSchema()

root
 |-- GEO_ID: string (nullable = true)
 |-- NAME: string (nullable = true)
 |-- state: integer (nullable = true)
 |-- county: integer (nullable = true)
 |-- tract: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- ed_total: double (nullable = true)
 |-- ed_fem: double (nullable = true)
 |-- ed_male: double (nullable = true)
 |-- trans_solo: double (nullable = true)
 |-- trans_pool: double (nullable = true)
 |-- trans_public: double (nullable = true)
 |-- trans_walk: double (nullable = true)
 |-- trans_remote: double (nullable = true)
 |-- perc_white: double (nullable = true)
 |-- perc_black: double (nullable = true)
 |-- med_hous_inc: integer (nullable = true)
 |-- per_cap_inc: integer (nullable = true)
 |-- med_age: double (nullable = true)
 |-- population: integer (nullable = true)



In [23]:
df_elig.count()

7672

In [24]:
census.count()

114935

In [25]:
percentile = census.groupby(['state','year']).agg(expr('percentile(med_hous_inc, array(0.20))')[0].alias('%20'))
percentile.show()

+-----+----+------------------+
|state|year|               %20|
+-----+----+------------------+
|   36|2015|           39460.0|
|   36|2014|           38847.2|
|    6|2016|           40757.2|
|   36|2012|           37869.6|
|   36|2018|           43737.4|
|    6|2018|           45625.0|
|   36|2017|           41378.0|
|   36|2010|           36822.6|
|    6|2017|           42819.0|
|    6|2013|           39538.4|
|    6|2014|           39536.6|
|    6|2012|           39756.0|
|   36|2013|           38371.8|
|    6|2015|           39604.6|
|    6|2010|           39970.8|
|    6|2011|40167.600000000006|
|   36|2016|           40365.0|
|   36|2011|           37569.6|
+-----+----+------------------+



In [27]:
percentile_ar = np.array(percentile.collect())

In [30]:
census_ar = np.array(census.filter(census.state == 6).collect())

In [31]:
def gentrified(state, year, geo_id, per_cap_inc):
    
    tract_summary = census_ar[(census_ar[:, 5] > year) & (census_ar[:, 5] <= year+3) & (census_ar[:, 0] == geo_id)][:, [5, 16, 17]]
    
    try:
        max_income = max(tract_summary[:, 2])
    
        if ((max_income - per_cap_inc) / per_cap_inc) >= .2:
            return int(1)
        else:
            for row in tract_summary:
                if row[1] >= percentile_ar[(percentile_ar[:, 0] == state) & (percentile_ar[:, 1] == year)][:, 2].item():
                    return int(1)
                else:
                    return int(0)
    except:
        return int(0)

In [32]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

In [10]:
states = [1,
 2,
 4,
 5,
 6,
 8,
 9,
 10,
 11,
 12,
 13,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 53,
 54,
 55,
 56]

In [34]:
df_list = []
for state in states:
    census_ar = np.array(census.filter(census.state == state).collect())
    temp = df_elig.filter(df_elig.state == state)
    labeler = udf(gentrified, IntegerType())
    df_list.append(temp.withColumn('gent', labeler('state', 'year', 'GEO_ID', 'per_cap_inc')))

In [35]:
df_list

[DataFrame[GEO_ID: string, NAME: string, state: int, county: int, tract: int, year: int, ed_total: double, ed_fem: double, ed_male: double, trans_solo: double, trans_pool: double, trans_public: double, trans_walk: double, trans_remote: double, perc_white: double, perc_black: double, med_hous_inc: int, per_cap_inc: int, med_age: double, population: int, gent: int],
 DataFrame[GEO_ID: string, NAME: string, state: int, county: int, tract: int, year: int, ed_total: double, ed_fem: double, ed_male: double, trans_solo: double, trans_pool: double, trans_public: double, trans_walk: double, trans_remote: double, perc_white: double, perc_black: double, med_hous_inc: int, per_cap_inc: int, med_age: double, population: int, gent: int]]

In [36]:
for ix, df in enumerate(df_list):
    if ix == 0:
        df_model = df
    else:
        df_model = df_model.union(df)

In [37]:
df_model.printSchema()

root
 |-- GEO_ID: string (nullable = true)
 |-- NAME: string (nullable = true)
 |-- state: integer (nullable = true)
 |-- county: integer (nullable = true)
 |-- tract: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- ed_total: double (nullable = true)
 |-- ed_fem: double (nullable = true)
 |-- ed_male: double (nullable = true)
 |-- trans_solo: double (nullable = true)
 |-- trans_pool: double (nullable = true)
 |-- trans_public: double (nullable = true)
 |-- trans_walk: double (nullable = true)
 |-- trans_remote: double (nullable = true)
 |-- perc_white: double (nullable = true)
 |-- perc_black: double (nullable = true)
 |-- med_hous_inc: integer (nullable = true)
 |-- per_cap_inc: integer (nullable = true)
 |-- med_age: double (nullable = true)
 |-- population: integer (nullable = true)
 |-- gent: integer (nullable = true)



In [38]:
df_model.count()

7672

In [39]:
df_model.filter(df_model.gent == 1).count()

2224

In [41]:
geo = spark.read.option('header', True).csv('data/geo*')

In [42]:
geo.show(5)

+---------+----------+---------+-----------+------+------------------+-------+----------+--------+--------+-----------+------------+
|STATEFP10|COUNTYFP10|TRACTCE10|    GEOID10|NAME10|        NAMELSAD10|MTFCC10|FUNCSTAT10| ALAND10|AWATER10| INTPTLAT10|  INTPTLON10|
+---------+----------+---------+-----------+------+------------------+-------+----------+--------+--------+-----------+------------+
|       06|       083|   002103|06083002103| 21.03|Census Tract 21.03|  G5020|         S| 2838200|    7603|+34.9306689|-120.4270588|
|       06|       083|   002402|06083002402| 24.02|Census Tract 24.02|  G5020|         S|16288573|   44468|+34.9287963|-120.4780833|
|       06|       083|   002102|06083002102| 21.02|Census Tract 21.02|  G5020|         S| 1352551|       0|+34.9421111|-120.4267767|
|       06|       083|   002010|06083002010| 20.10|Census Tract 20.10|  G5020|         S| 2417990|       0|+34.8714281|-120.4100285|
|       06|       083|   002009|06083002009| 20.09|Census Tract 20.09

In [43]:
geo.printSchema()

root
 |-- STATEFP10: string (nullable = true)
 |-- COUNTYFP10: string (nullable = true)
 |-- TRACTCE10: string (nullable = true)
 |-- GEOID10: string (nullable = true)
 |-- NAME10: string (nullable = true)
 |-- NAMELSAD10: string (nullable = true)
 |-- MTFCC10: string (nullable = true)
 |-- FUNCSTAT10: string (nullable = true)
 |-- ALAND10: string (nullable = true)
 |-- AWATER10: string (nullable = true)
 |-- INTPTLAT10: string (nullable = true)
 |-- INTPTLON10: string (nullable = true)



In [44]:
geo.count()

12976

In [45]:
df_model.printSchema()

root
 |-- GEO_ID: string (nullable = true)
 |-- NAME: string (nullable = true)
 |-- state: integer (nullable = true)
 |-- county: integer (nullable = true)
 |-- tract: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- ed_total: double (nullable = true)
 |-- ed_fem: double (nullable = true)
 |-- ed_male: double (nullable = true)
 |-- trans_solo: double (nullable = true)
 |-- trans_pool: double (nullable = true)
 |-- trans_public: double (nullable = true)
 |-- trans_walk: double (nullable = true)
 |-- trans_remote: double (nullable = true)
 |-- perc_white: double (nullable = true)
 |-- perc_black: double (nullable = true)
 |-- med_hous_inc: integer (nullable = true)
 |-- per_cap_inc: integer (nullable = true)
 |-- med_age: double (nullable = true)
 |-- population: integer (nullable = true)
 |-- gent: integer (nullable = true)



In [46]:
df_model = df_model.join(geo.select(['GEOID10', 'INTPTLAT10', 'INTPTLON10', 'ALAND10']), df_model.GEO_ID == geo.GEOID10, how='left')

In [47]:
df_model.printSchema()

root
 |-- GEO_ID: string (nullable = true)
 |-- NAME: string (nullable = true)
 |-- state: integer (nullable = true)
 |-- county: integer (nullable = true)
 |-- tract: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- ed_total: double (nullable = true)
 |-- ed_fem: double (nullable = true)
 |-- ed_male: double (nullable = true)
 |-- trans_solo: double (nullable = true)
 |-- trans_pool: double (nullable = true)
 |-- trans_public: double (nullable = true)
 |-- trans_walk: double (nullable = true)
 |-- trans_remote: double (nullable = true)
 |-- perc_white: double (nullable = true)
 |-- perc_black: double (nullable = true)
 |-- med_hous_inc: integer (nullable = true)
 |-- per_cap_inc: integer (nullable = true)
 |-- med_age: double (nullable = true)
 |-- population: integer (nullable = true)
 |-- gent: integer (nullable = true)
 |-- GEOID10: string (nullable = true)
 |-- INTPTLAT10: string (nullable = true)
 |-- INTPTLON10: string (nullable = true)
 |-- ALAND10: string (n

In [48]:
census.count()

114935

In [49]:
census.printSchema()

root
 |-- GEO_ID: string (nullable = true)
 |-- NAME: string (nullable = true)
 |-- state: integer (nullable = true)
 |-- county: integer (nullable = true)
 |-- tract: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- ed_total: double (nullable = true)
 |-- ed_fem: double (nullable = true)
 |-- ed_male: double (nullable = true)
 |-- trans_solo: double (nullable = true)
 |-- trans_pool: double (nullable = true)
 |-- trans_public: double (nullable = true)
 |-- trans_walk: double (nullable = true)
 |-- trans_remote: double (nullable = true)
 |-- perc_white: double (nullable = true)
 |-- perc_black: double (nullable = true)
 |-- med_hous_inc: integer (nullable = true)
 |-- per_cap_inc: integer (nullable = true)
 |-- med_age: double (nullable = true)
 |-- population: integer (nullable = true)



In [50]:
census = census.join(geo.select(['GEOID10', 'INTPTLAT10', 'INTPTLON10', 'ALAND10']), census.GEO_ID == geo.GEOID10, how='left')

In [51]:
census.count()

114935

In [52]:
census.printSchema()

root
 |-- GEO_ID: string (nullable = true)
 |-- NAME: string (nullable = true)
 |-- state: integer (nullable = true)
 |-- county: integer (nullable = true)
 |-- tract: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- ed_total: double (nullable = true)
 |-- ed_fem: double (nullable = true)
 |-- ed_male: double (nullable = true)
 |-- trans_solo: double (nullable = true)
 |-- trans_pool: double (nullable = true)
 |-- trans_public: double (nullable = true)
 |-- trans_walk: double (nullable = true)
 |-- trans_remote: double (nullable = true)
 |-- perc_white: double (nullable = true)
 |-- perc_black: double (nullable = true)
 |-- med_hous_inc: integer (nullable = true)
 |-- per_cap_inc: integer (nullable = true)
 |-- med_age: double (nullable = true)
 |-- population: integer (nullable = true)
 |-- GEOID10: string (nullable = true)
 |-- INTPTLAT10: string (nullable = true)
 |-- INTPTLON10: string (nullable = true)
 |-- ALAND10: string (nullable = true)



In [53]:
df_model.count()

7672

In [54]:
census = census.na.drop()
df_model = df_model.na.drop()

In [55]:
from pyspark.sql.types import *

In [56]:
census = census.withColumn('INTPTLAT10', census.INTPTLAT10.cast(FloatType()))
census = census.withColumn('INTPTLON10', census.INTPTLON10.cast(FloatType()))
census = census.withColumn('ALAND10', census.ALAND10.cast(IntegerType()))


In [57]:
df_model = df_model.withColumn('INTPTLAT10', df_model.INTPTLAT10.cast(FloatType()))
df_model = df_model.withColumn('INTPTLON10', df_model.INTPTLON10.cast(FloatType()))
df_model = df_model.withColumn('ALAND10', df_model.ALAND10.cast(IntegerType()))

In [58]:
df_model.printSchema()

root
 |-- GEO_ID: string (nullable = true)
 |-- NAME: string (nullable = true)
 |-- state: integer (nullable = true)
 |-- county: integer (nullable = true)
 |-- tract: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- ed_total: double (nullable = true)
 |-- ed_fem: double (nullable = true)
 |-- ed_male: double (nullable = true)
 |-- trans_solo: double (nullable = true)
 |-- trans_pool: double (nullable = true)
 |-- trans_public: double (nullable = true)
 |-- trans_walk: double (nullable = true)
 |-- trans_remote: double (nullable = true)
 |-- perc_white: double (nullable = true)
 |-- perc_black: double (nullable = true)
 |-- med_hous_inc: integer (nullable = true)
 |-- per_cap_inc: integer (nullable = true)
 |-- med_age: double (nullable = true)
 |-- population: integer (nullable = true)
 |-- gent: integer (nullable = true)
 |-- GEOID10: string (nullable = true)
 |-- INTPTLAT10: float (nullable = true)
 |-- INTPTLON10: float (nullable = true)
 |-- ALAND10: integer (nu

In [59]:
def tract_dist(lat1, long1, lat2, long2):
    '''
    

    Parameters
    ----------
    lat1 : float
        Internal latitude for first tract
    long1 : float
        Internal longitude for first tract
    lat2 : float
        Internal latitude for second tract
    long2 : float
        Internal longitude for second tract

    Returns
    -------
    float
        Euclidean distance between both tracts' internal points

    '''
    return np.sqrt((lat1 - lat2)**2 + (long1- long2)**2)

In [60]:
census.printSchema()

root
 |-- GEO_ID: string (nullable = true)
 |-- NAME: string (nullable = true)
 |-- state: integer (nullable = true)
 |-- county: integer (nullable = true)
 |-- tract: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- ed_total: double (nullable = true)
 |-- ed_fem: double (nullable = true)
 |-- ed_male: double (nullable = true)
 |-- trans_solo: double (nullable = true)
 |-- trans_pool: double (nullable = true)
 |-- trans_public: double (nullable = true)
 |-- trans_walk: double (nullable = true)
 |-- trans_remote: double (nullable = true)
 |-- perc_white: double (nullable = true)
 |-- perc_black: double (nullable = true)
 |-- med_hous_inc: integer (nullable = true)
 |-- per_cap_inc: integer (nullable = true)
 |-- med_age: double (nullable = true)
 |-- population: integer (nullable = true)
 |-- GEOID10: string (nullable = true)
 |-- INTPTLAT10: float (nullable = true)
 |-- INTPTLON10: float (nullable = true)
 |-- ALAND10: integer (nullable = true)



In [61]:
geo_array = np.array(census.select(['GEOID10', 'INTPTLAT10', 'INTPTLON10', 'year']).collect())

In [62]:
type(str(geo_array[0,0]))

str

In [63]:
float(geo_array[0, 1])

37.34346389770508

In [64]:
int(geo_array[0, 3])

2018

In [65]:
def neighbors(geo_id, lat, lon, year):
    
    distances = []
    
    for geo_id2, lat2, lon2, year2 in geo_array:
        if int(year2) == year:
            distances.append((geo_id2, tract_dist(lat, lon, float(lat2), float(lon2))))
            
    distances.sort(key = lambda x: x[1])
    return str(distances[n+1][0])
    

In [67]:
num_neighbors = 2

In [68]:
list(range(num_neighbors))

[0, 1]

In [69]:
for n in range(num_neighbors):

    df_list = []
    for state in states:
        geo_array = np.array(census.filter(census.state == state).select(['GEOID10', 'INTPTLAT10', 'INTPTLON10', 'year']).collect())
        temp = df_model.filter(df_model.state == state)
        neighbor_function = udf(neighbors, StringType())
        df_list.append(temp.withColumn('neighbor'+str(n), neighbor_function('GEO_ID', 'INTPTLAT10', 'INTPTLON10', 'year')))

    for ix, dfc in enumerate(df_list):
        if ix == 0:
            df_model = dfc
        else:
            df_model = df_model.union(dfc)

In [70]:
df_model.printSchema()

root
 |-- GEO_ID: string (nullable = true)
 |-- NAME: string (nullable = true)
 |-- state: integer (nullable = true)
 |-- county: integer (nullable = true)
 |-- tract: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- ed_total: double (nullable = true)
 |-- ed_fem: double (nullable = true)
 |-- ed_male: double (nullable = true)
 |-- trans_solo: double (nullable = true)
 |-- trans_pool: double (nullable = true)
 |-- trans_public: double (nullable = true)
 |-- trans_walk: double (nullable = true)
 |-- trans_remote: double (nullable = true)
 |-- perc_white: double (nullable = true)
 |-- perc_black: double (nullable = true)
 |-- med_hous_inc: integer (nullable = true)
 |-- per_cap_inc: integer (nullable = true)
 |-- med_age: double (nullable = true)
 |-- population: integer (nullable = true)
 |-- gent: integer (nullable = true)
 |-- GEOID10: string (nullable = true)
 |-- INTPTLAT10: float (nullable = true)
 |-- INTPTLON10: float (nullable = true)
 |-- ALAND10: integer (nu

In [71]:
df_model.show()

+-----------+--------------------+-----+------+------+----+--------------------+--------------------+--------------------+-------------------+--------------------+-------------------+--------------------+--------------------+-------------------+-------------------+------------+-----------+-------+----------+----+-----------+----------+-----------+-------+-----------+-----------+
|     GEO_ID|                NAME|state|county| tract|year|            ed_total|              ed_fem|             ed_male|         trans_solo|          trans_pool|       trans_public|          trans_walk|        trans_remote|         perc_white|         perc_black|med_hous_inc|per_cap_inc|med_age|population|gent|    GEOID10|INTPTLAT10| INTPTLON10|ALAND10|  neighbor0|  neighbor1|
+-----------+--------------------+-----+------+------+----+--------------------+--------------------+--------------------+-------------------+--------------------+-------------------+--------------------+--------------------+-----------

In [72]:
df_model.count()

7666

In [73]:
df_model.printSchema()

root
 |-- GEO_ID: string (nullable = true)
 |-- NAME: string (nullable = true)
 |-- state: integer (nullable = true)
 |-- county: integer (nullable = true)
 |-- tract: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- ed_total: double (nullable = true)
 |-- ed_fem: double (nullable = true)
 |-- ed_male: double (nullable = true)
 |-- trans_solo: double (nullable = true)
 |-- trans_pool: double (nullable = true)
 |-- trans_public: double (nullable = true)
 |-- trans_walk: double (nullable = true)
 |-- trans_remote: double (nullable = true)
 |-- perc_white: double (nullable = true)
 |-- perc_black: double (nullable = true)
 |-- med_hous_inc: integer (nullable = true)
 |-- per_cap_inc: integer (nullable = true)
 |-- med_age: double (nullable = true)
 |-- population: integer (nullable = true)
 |-- gent: integer (nullable = true)
 |-- GEOID10: string (nullable = true)
 |-- INTPTLAT10: float (nullable = true)
 |-- INTPTLON10: float (nullable = true)
 |-- ALAND10: integer (nu

In [74]:
census.printSchema()

root
 |-- GEO_ID: string (nullable = true)
 |-- NAME: string (nullable = true)
 |-- state: integer (nullable = true)
 |-- county: integer (nullable = true)
 |-- tract: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- ed_total: double (nullable = true)
 |-- ed_fem: double (nullable = true)
 |-- ed_male: double (nullable = true)
 |-- trans_solo: double (nullable = true)
 |-- trans_pool: double (nullable = true)
 |-- trans_public: double (nullable = true)
 |-- trans_walk: double (nullable = true)
 |-- trans_remote: double (nullable = true)
 |-- perc_white: double (nullable = true)
 |-- perc_black: double (nullable = true)
 |-- med_hous_inc: integer (nullable = true)
 |-- per_cap_inc: integer (nullable = true)
 |-- med_age: double (nullable = true)
 |-- population: integer (nullable = true)
 |-- GEOID10: string (nullable = true)
 |-- INTPTLAT10: float (nullable = true)
 |-- INTPTLON10: float (nullable = true)
 |-- ALAND10: integer (nullable = true)



In [75]:
df_merge = census.select(['GEO_ID',
                   'year',
                   'ed_total',
                   'trans_solo',
                   'trans_pool',
                   'trans_public',
                   'trans_walk',
                   'trans_remote',
                   'perc_white',
                   'perc_black',
                   'med_hous_inc',
                   'per_cap_inc',
                   'med_age',
                   'population',
                   'ALAND10'])

In [81]:
dfj = df_merge.select(col('GEO_ID').alias(str(n)+'_GEO_ID'),
                               col('year').alias(str(n)+'_year'),
                               col('ed_total').alias(str(n)+'_ed_total'),
                               col('trans_solo').alias(str(n)+'_trans_solo'),
                               col('trans_pool').alias(str(n)+'_trans_pool'),
                               col('trans_public').alias(str(n)+'_trans_public'),
                               col('trans_walk').alias(str(n)+'_trans_walk'),
                               col('trans_remote').alias(str(n)+'_trans_remote'),
                               col('perc_white').alias(str(n)+'_perc_white'),
                               col('perc_black').alias(str(n)+'_perc_black'),
                               col('med_hous_inc').alias(str(n)+'_med_house_inc'),
                               col('per_cap_inc').alias(str(n)+'_per_cap_inc'),
                               col('med_age').alias(str(n)+'_med_age'),
                               col('population').alias(str(n)+'_population'),
                               col('ALAND10').alias(str(n)+'_land_area'))

In [82]:
dfj.count()

114824

In [83]:
df_model.count()

7666

In [84]:
df_model.printSchema()

root
 |-- GEO_ID: string (nullable = true)
 |-- NAME: string (nullable = true)
 |-- state: integer (nullable = true)
 |-- county: integer (nullable = true)
 |-- tract: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- ed_total: double (nullable = true)
 |-- ed_fem: double (nullable = true)
 |-- ed_male: double (nullable = true)
 |-- trans_solo: double (nullable = true)
 |-- trans_pool: double (nullable = true)
 |-- trans_public: double (nullable = true)
 |-- trans_walk: double (nullable = true)
 |-- trans_remote: double (nullable = true)
 |-- perc_white: double (nullable = true)
 |-- perc_black: double (nullable = true)
 |-- med_hous_inc: integer (nullable = true)
 |-- per_cap_inc: integer (nullable = true)
 |-- med_age: double (nullable = true)
 |-- population: integer (nullable = true)
 |-- gent: integer (nullable = true)
 |-- GEOID10: string (nullable = true)
 |-- INTPTLAT10: float (nullable = true)
 |-- INTPTLON10: float (nullable = true)
 |-- ALAND10: integer (nu

In [85]:
dfj.printSchema()

root
 |-- 1_GEO_ID: string (nullable = true)
 |-- 1_year: integer (nullable = true)
 |-- 1_ed_total: double (nullable = true)
 |-- 1_trans_solo: double (nullable = true)
 |-- 1_trans_pool: double (nullable = true)
 |-- 1_trans_public: double (nullable = true)
 |-- 1_trans_walk: double (nullable = true)
 |-- 1_trans_remote: double (nullable = true)
 |-- 1_perc_white: double (nullable = true)
 |-- 1_perc_black: double (nullable = true)
 |-- 1_med_house_inc: integer (nullable = true)
 |-- 1_per_cap_inc: integer (nullable = true)
 |-- 1_med_age: double (nullable = true)
 |-- 1_population: integer (nullable = true)
 |-- 1_land_area: integer (nullable = true)



In [86]:
df_model.join(dfj, [df_model['neighbor'+str(n)] == dfj[str(n)+'_GEO_ID'], df_model.year == dfj[str(n)+'_year']], how='left').count()

7666

In [79]:
for n in range(num_neighbors):
    
    dfj = df_merge.select(col('GEO_ID').alias(str(n)+'_GEO_ID'),
                               col('year').alias(str(n)+'_year'),
                               col('ed_total').alias(str(n)+'_ed_total'),
                               col('trans_solo').alias(str(n)+'_trans_solo'),
                               col('trans_pool').alias(str(n)+'_trans_pool'),
                               col('trans_public').alias(str(n)+'_trans_public'),
                               col('trans_walk').alias(str(n)+'_trans_walk'),
                               col('trans_remote').alias(str(n)+'_trans_remote'),
                               col('perc_white').alias(str(n)+'_perc_white'),
                               col('perc_black').alias(str(n)+'_perc_black'),
                               col('med_hous_inc').alias(str(n)+'_med_house_inc'),
                               col('per_cap_inc').alias(str(n)+'_per_cap_inc'),
                               col('med_age').alias(str(n)+'_med_age'),
                               col('population').alias(str(n)+'_population'),
                               col('ALAND10').alias(str(n)+'_land_area'))
    
    df_model = df_model.join(dfj, (df_model['neighbor'+str(n)] == dfj[str(n)+'_GEO_ID']) & (df_model.year == dfj[str(n)+'_year']), how='left')
    

In [80]:
df_model.printSchema()

root
 |-- GEO_ID: string (nullable = true)
 |-- NAME: string (nullable = true)
 |-- state: integer (nullable = true)
 |-- county: integer (nullable = true)
 |-- tract: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- ed_total: double (nullable = true)
 |-- ed_fem: double (nullable = true)
 |-- ed_male: double (nullable = true)
 |-- trans_solo: double (nullable = true)
 |-- trans_pool: double (nullable = true)
 |-- trans_public: double (nullable = true)
 |-- trans_walk: double (nullable = true)
 |-- trans_remote: double (nullable = true)
 |-- perc_white: double (nullable = true)
 |-- perc_black: double (nullable = true)
 |-- med_hous_inc: integer (nullable = true)
 |-- per_cap_inc: integer (nullable = true)
 |-- med_age: double (nullable = true)
 |-- population: integer (nullable = true)
 |-- gent: integer (nullable = true)
 |-- GEOID10: string (nullable = true)
 |-- INTPTLAT10: float (nullable = true)
 |-- INTPTLON10: float (nullable = true)
 |-- ALAND10: integer (nu

In [89]:
df_model.show()

+-----------+--------------------+-----+------+------+----+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+------------+-----------+-------+----------+----+-----------+----------+-----------+----------+-----------+-----------+-----------+------+-------------------+-------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+---------------+-------------+---------+------------+-----------+-----------+------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+---------------+-------------+---------+------------+-----------+
|     GEO_ID|                NAME|state|county| tract|year|            ed_total|              ed_fem|             

In [90]:
df_model.count()

7666

In [81]:
neighbors = [str(x) for x in range(num_neighbors)]

In [82]:
neighbors

['0', '1']

In [93]:
df_safe = df_model

In [87]:
df_model.withColumn('n_ed_total', sum([df_model[n+'_ed_total']*df_model[n+'_population'] 
                                  for n in neighbors]) / sum([df_model[n+'_population'] for n in neighbors])).show()

+-----------+--------------------+-----+------+------+----+-------------------+-------------------+--------------------+------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+------------+-----------+-------+----------+----+-----------+----------+----------+--------+-----------+-----------+-----------+------+-------------------+------------------+-------------------+-------------------+--------------------+--------------------+-------------------+------------------+---------------+-------------+---------+------------+-----------+-----------+------+-------------------+-------------------+--------------------+-------------------+--------------------+--------------------+-------------------+--------------------+---------------+-------------+---------+------------+-----------+-------------------+
|     GEO_ID|                NAME|state|county| tract|year|           ed_total|             ed_fem|             e

In [83]:
    
# Lets aggregate features for neighboring tracts by taking a weighted average based on population
df_model = df_model.withColumn('n_ed_total', sum([df_model[n+'_ed_total']*df_model[n+'_population'] 
                                  for n in neighbors]) / sum([df_model[n+'_population'] for n in neighbors]))
    
df_model = df_model.withColumn('n_trans_solo', sum([df_model[n+'_trans_solo']*df_model[n+'_population'] 
                                     for n in neighbors]) / sum([df_model[n+'_population'] for n in neighbors]))
    
df_model = df_model.withColumn('n_trans_pool', sum([df_model[n+'_trans_pool']*df_model[n+'_population'] 
                                     for n in neighbors]) / sum([df_model[n+'_population'] for n in neighbors]))
    
df_model = df_model.withColumn('n_trans_public', sum([df_model[n+'_trans_public']*df_model[n+'_population'] 
                                     for n in neighbors]) / sum([df_model[n+'_population'] for n in neighbors]))
    
df_model = df_model.withColumn('n_trans_walk', sum([df_model[n+'_trans_walk']*df_model[n+'_population'] 
                                     for n in neighbors]) / sum([df_model[n+'_population'] for n in neighbors]))
    
df_model = df_model.withColumn('n_trans_remote', sum([df_model[n+'_trans_remote']*df_model[n+'_population'] 
                                     for n in neighbors]) / sum([df_model[n+'_population'] for n in neighbors]))
    
df_model = df_model.withColumn('n_perc_white', sum([df_model[n+'_perc_white']*df_model[n+'_population'] 
                                     for n in neighbors]) / sum([df_model[n+'_population'] for n in neighbors]))
    
df_model = df_model.withColumn('n_perc_black', sum([df_model[n+'_perc_black']*df_model[n+'_population'] 
                                     for n in neighbors]) / sum([df_model[n+'_population'] for n in neighbors]))
    
df_model = df_model.withColumn('n_per_cap_inc', sum([df_model[n+'_per_cap_inc']*df_model[n+'_population'] 
                                     for n in neighbors]) / sum([df_model[n+'_population'] for n in neighbors]))

df_model = df_model.withColumn('n_med_house_inc', sum([df_model[n+'_med_house_inc']*df_model[n+'_population'] 
                                     for n in neighbors]) / sum([df_model[n+'_population'] for n in neighbors]))

df_model = df_model.withColumn('n_med_age', sum([df_model[n+'_med_age']*df_model[n+'_population'] 
                                     for n in neighbors]) / sum([df_model[n+'_population'] for n in neighbors]))
    
df_model = df_model.withColumn('n_population', sum([df_model[n+'_population'] for n in neighbors])) # created a feature containing population total of all neighboring tracts
    
df_model = df_model.withColumn('n_land_area', sum([df_model[n+'_land_area'] for n in neighbors])) # created a feature containing total land area of neighboring tracts
    
    

In [84]:
df_model.printSchema()

root
 |-- GEO_ID: string (nullable = true)
 |-- NAME: string (nullable = true)
 |-- state: integer (nullable = true)
 |-- county: integer (nullable = true)
 |-- tract: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- ed_total: double (nullable = true)
 |-- ed_fem: double (nullable = true)
 |-- ed_male: double (nullable = true)
 |-- trans_solo: double (nullable = true)
 |-- trans_pool: double (nullable = true)
 |-- trans_public: double (nullable = true)
 |-- trans_walk: double (nullable = true)
 |-- trans_remote: double (nullable = true)
 |-- perc_white: double (nullable = true)
 |-- perc_black: double (nullable = true)
 |-- med_hous_inc: integer (nullable = true)
 |-- per_cap_inc: integer (nullable = true)
 |-- med_age: double (nullable = true)
 |-- population: integer (nullable = true)
 |-- gent: integer (nullable = true)
 |-- GEOID10: string (nullable = true)
 |-- INTPTLAT10: float (nullable = true)
 |-- INTPTLON10: float (nullable = true)
 |-- ALAND10: integer (nu

In [85]:
df_model.schema.names[:25+num_neighbors]

['GEO_ID',
 'NAME',
 'state',
 'county',
 'tract',
 'year',
 'ed_total',
 'ed_fem',
 'ed_male',
 'trans_solo',
 'trans_pool',
 'trans_public',
 'trans_walk',
 'trans_remote',
 'perc_white',
 'perc_black',
 'med_hous_inc',
 'per_cap_inc',
 'med_age',
 'population',
 'gent',
 'GEOID10',
 'INTPTLAT10',
 'INTPTLON10',
 'ALAND10',
 'neighbor0',
 'neighbor1']

In [86]:
df_model.schema.names[25+num_neighbors+(15*num_neighbors):]

['n_ed_total',
 'n_trans_solo',
 'n_trans_pool',
 'n_trans_public',
 'n_trans_walk',
 'n_trans_remote',
 'n_perc_white',
 'n_perc_black',
 'n_per_cap_inc',
 'n_med_house_inc',
 'n_med_age',
 'n_population',
 'n_land_area']

In [87]:
final_schema = df_model.schema.names[5:21] + df_model.schema.names[24:25] + df_model.schema.names[25+num_neighbors+(15*num_neighbors):]

In [88]:
final_schema

['year',
 'ed_total',
 'ed_fem',
 'ed_male',
 'trans_solo',
 'trans_pool',
 'trans_public',
 'trans_walk',
 'trans_remote',
 'perc_white',
 'perc_black',
 'med_hous_inc',
 'per_cap_inc',
 'med_age',
 'population',
 'gent',
 'ALAND10',
 'n_ed_total',
 'n_trans_solo',
 'n_trans_pool',
 'n_trans_public',
 'n_trans_walk',
 'n_trans_remote',
 'n_perc_white',
 'n_perc_black',
 'n_per_cap_inc',
 'n_med_house_inc',
 'n_med_age',
 'n_population',
 'n_land_area']

In [89]:
dff = df_model.select(final_schema)

In [90]:
dff.printSchema()

root
 |-- year: integer (nullable = true)
 |-- ed_total: double (nullable = true)
 |-- ed_fem: double (nullable = true)
 |-- ed_male: double (nullable = true)
 |-- trans_solo: double (nullable = true)
 |-- trans_pool: double (nullable = true)
 |-- trans_public: double (nullable = true)
 |-- trans_walk: double (nullable = true)
 |-- trans_remote: double (nullable = true)
 |-- perc_white: double (nullable = true)
 |-- perc_black: double (nullable = true)
 |-- med_hous_inc: integer (nullable = true)
 |-- per_cap_inc: integer (nullable = true)
 |-- med_age: double (nullable = true)
 |-- population: integer (nullable = true)
 |-- gent: integer (nullable = true)
 |-- ALAND10: integer (nullable = true)
 |-- n_ed_total: double (nullable = true)
 |-- n_trans_solo: double (nullable = true)
 |-- n_trans_pool: double (nullable = true)
 |-- n_trans_public: double (nullable = true)
 |-- n_trans_walk: double (nullable = true)
 |-- n_trans_remote: double (nullable = true)
 |-- n_perc_white: double (nu

In [91]:
dff.show()

+----+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+------------+-----------+-------+----------+----+----------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+------------------+------------------+------------------+------------+-----------+
|year|            ed_total|              ed_fem|             ed_male|         trans_solo|          trans_pool|        trans_public|          trans_walk|        trans_remote|         perc_white|          perc_black|med_hous_inc|per_cap_inc|med_age|population|gent|   ALAND10|          n_ed_total|       n_trans_solo|        n_trans_pool|      n_trans_public|        n_trans_walk|      n_trans_remote|       n_perc_white|        n_perc_black|     n_per_cap_inc|   n_med_house_inc

In [103]:
dff.count()

7666

In [92]:
dff.schema.names

['year',
 'ed_total',
 'ed_fem',
 'ed_male',
 'trans_solo',
 'trans_pool',
 'trans_public',
 'trans_walk',
 'trans_remote',
 'perc_white',
 'perc_black',
 'med_hous_inc',
 'per_cap_inc',
 'med_age',
 'population',
 'gent',
 'ALAND10',
 'n_ed_total',
 'n_trans_solo',
 'n_trans_pool',
 'n_trans_public',
 'n_trans_walk',
 'n_trans_remote',
 'n_perc_white',
 'n_perc_black',
 'n_per_cap_inc',
 'n_med_house_inc',
 'n_med_age',
 'n_population',
 'n_land_area']

In [93]:
features = ['year',
            'ed_total',
            'trans_solo',
            'trans_pool',
            'trans_public',
            'trans_walk',
            'trans_remote',
            'perc_white',
            'perc_black',
            'med_hous_inc',
            'per_cap_inc',
            'med_age',
            'population',
            'ALAND10',
            'n_ed_total',
            'n_trans_solo',
            'n_trans_pool',
            'n_trans_public',
            'n_trans_walk',
            'n_trans_remote',
            'n_perc_white',
            'n_perc_black',
            'n_per_cap_inc',
            'n_med_house_inc',
            'n_med_age',
            'n_population',
            'n_land_area']

In [94]:
features

['year',
 'ed_total',
 'trans_solo',
 'trans_pool',
 'trans_public',
 'trans_walk',
 'trans_remote',
 'perc_white',
 'perc_black',
 'med_hous_inc',
 'per_cap_inc',
 'med_age',
 'population',
 'ALAND10',
 'n_ed_total',
 'n_trans_solo',
 'n_trans_pool',
 'n_trans_public',
 'n_trans_walk',
 'n_trans_remote',
 'n_perc_white',
 'n_perc_black',
 'n_per_cap_inc',
 'n_med_house_inc',
 'n_med_age',
 'n_population',
 'n_land_area']

In [98]:
dff = dff.na.drop()

In [113]:
dff = spark.createDataFrame(dff.collect())

In [114]:
dff.count()

7583

In [115]:
dff.show()

+----+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+------------+-----------+-------+----------+----+----------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+------------------+------------------+------------------+------------+-----------+
|year|            ed_total|              ed_fem|             ed_male|         trans_solo|          trans_pool|        trans_public|          trans_walk|        trans_remote|         perc_white|          perc_black|med_hous_inc|per_cap_inc|med_age|population|gent|   ALAND10|          n_ed_total|       n_trans_solo|        n_trans_pool|      n_trans_public|        n_trans_walk|      n_trans_remote|       n_perc_white|        n_perc_black|     n_per_cap_inc|   n_med_house_inc

In [117]:
dff.filter(dff.gent == 1).count()

2196

In [121]:
train, test = dff.randomSplit([.8, .2], seed = 42)

In [122]:
train.count()

6093

In [123]:
train.filter(train.gent == 0).count()

4329

In [124]:
train.filter(train.gent == 1).count()

1764

In [125]:
train = train.filter(train.gent == 1).union(train.filter(train.gent == 0).sample(train.filter(train.gent == 1).count()/train.filter(train.gent == 0).count()))

In [126]:
train.count()

3552

In [128]:
train.filter(train.gent == 0).count()

1788

In [129]:
from pyspark.ml.feature import VectorAssembler

In [130]:
assembler = VectorAssembler(inputCols=features, outputCol='features')
train_pack = assembler.transform(train)
test_pack = assembler.transform(test)

In [131]:
from pyspark.ml.feature import MinMaxScaler

minmaxscale = MinMaxScaler(inputCol='features', outputCol='features_scaled')
minmaxscale = minmaxscale.fit(train_pack)

train_pack = minmaxscale.transform(train_pack)
test_pack  = minmaxscale.transform(test_pack)


In [132]:
from pyspark.ml.classification import RandomForestClassifier, GBTClassifier, LogisticRegression

In [140]:
rf = RandomForestClassifier(labelCol='gent',
                            featuresCol='features',
                            predictionCol='prediction')

rf_model = rf.fit(train_pack)
rf_pred = rf_model.transform(test_pack)


In [135]:
lr = LogisticRegression(labelCol='gent', 
                        featuresCol='features_scaled',
                        predictionCol='prediction')

lr_model = lr.fit(train_pack)
lr_pred = lr_model.transform(test_pack)

In [105]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics

In [141]:
preds_and_labels = rf_pred.select(['prediction','gent']).withColumn('label', col('gent').cast(FloatType())).orderBy('prediction')

#select only prediction and label columns
preds_and_labels = preds_and_labels.select(['prediction','label'])

metrics = MulticlassMetrics(preds_and_labels.rdd.map(tuple))

print(metrics.confusionMatrix().toArray())


[[685. 373.]
 [157. 275.]]


In [147]:

print(metrics.precision(0))

0.8135391923990499


In [146]:
print(metrics.recall(0))

0.6474480151228733


In [136]:
preds_and_labels = lr_pred.select(['prediction','gent']).withColumn('label', col('gent').cast(FloatType())).orderBy('prediction')

#select only prediction and label columns
preds_and_labels = preds_and_labels.select(['prediction','label'])

metrics = MulticlassMetrics(preds_and_labels.rdd.map(tuple))

print(metrics.confusionMatrix().toArray())


[[702. 356.]
 [172. 260.]]


In [122]:
precision = BinaryClassificationEvaluator(labelCol='gent', 
                                              predictionCol='prediction',
                                              metricName='weightedPrecision')

recall = BinaryClassificationEvaluator(labelCol='gent', 
                                           predictionCol='prediction',
                                           metricName='weightedRecall')

f1 = MulticlassClassificationEvaluator(labelCol='gent', 
                                       predictionCol='prediction',
                                       metricName='f1')

areaROC = BinaryClassificationEvaluator(labelCol='gent',
                                        rawPredictionCol='prediction',
                                        metricName='areaUnderROC')

In [123]:
prec = precision.evaluate(lr_pred)

In [124]:
prec

0.6694168336158628

In [125]:
rec = recall.evaluate(lr_pred)

In [126]:
rec

0.7203883495145631